# Assignment 4: Decision Tree Implementation (Python Only)

<b>Name</b>:Matthew Carson <br>
<b>Email</b>:mcarso15@uncc.edu <br>

Decision trees are usually not used for prediction but for data interpretation, understanding interactions and behavior. Decision trees are very good at approximating highly non linear models with complex interactions and that's where you will get the most bang for your buck.

<b>Advantages of Decision Tree</b>:
- Advantage 1: Decision trees require relatively little effort from users for data preparation
- Advantage 2: Nonlinear relationships between parameters do not affect tree performance

<b>Disadvantages of Decision Tree</b>:
- Disadvantage 1: Prone to overfitting
- Disadvantage 2: Optimal decision tree is NP-complete problem

<b>Implementation</b>

- Before starting to code it is highly imperative to go through class notes and understand the working of a decision tree classifier and what is meant by CART (Classification and Regression Trees).
- Please complete the <font color = 'red'>TODO</font> sections in the notebook.
- Please include References at the end of your notebook



# Data

In [6]:
#importing respective libraries and setting up the enviornment

'''data working libraries'''
import pandas as pd
import numpy as np

'''data visualisation libraries'''
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
py.offline.init_notebook_mode(connected=True)
%matplotlib inline

In [8]:
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
#load dataset
# You may download the dataset from https://www.kaggle.com/uciml/pima-indians-diabetes-database#diabetes.csv
dataset = pd.read_csv("diabetes.csv", header=None, names = col_names)

In [9]:
dataset.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
1,6,148,72,35,0,33.6,0.627,50,1
2,1,85,66,29,0,26.6,0.351,31,0
3,8,183,64,0,0,23.3,0.672,32,1
4,1,89,66,23,94,28.1,0.167,21,0


In [10]:
dataset.label.replace(0,'Non-diabetic',inplace=True)
dataset.label.replace(1.0,'Diabetic',inplace=True)

In [11]:
feature_cols = ['pregnant', 'insulin', 'bmi', 'age','glucose','bp','pedigree']
header=feature_cols + ['label']
X = dataset[feature_cols] # Features
y = dataset.label # Target variable
X = np.float64(X[1:])
y = np.int64(y[1:])

In [12]:
# Dataframe to numpy array...
data=np.c_[X,y]
data

array([[  6.   ,   0.   ,  33.6  , ...,  72.   ,   0.627,   1.   ],
       [  1.   ,   0.   ,  26.6  , ...,  66.   ,   0.351,   0.   ],
       [  8.   ,   0.   ,  23.3  , ...,  64.   ,   0.672,   1.   ],
       ...,
       [  5.   , 112.   ,  26.2  , ...,  72.   ,   0.245,   0.   ],
       [  1.   ,   0.   ,  30.1  , ...,  60.   ,   0.349,   1.   ],
       [  1.   ,   0.   ,  30.4  , ...,  70.   ,   0.315,   0.   ]])

In [13]:
data.shape

(768, 8)

### <font color = 'red'>TODO</font>: Please provide a short description of the data and the aim of this assignment. You may refer to the following link: https://www.kaggle.com/uciml/pima-indians-diabetes-database#diabetes.csv

This dataset contains information on 768 females who are at least 21 years old and of Pima Indian heritage that may be used to predict whether or not they have diabetes. The aim of this assignment is to use Decision Tree to attempt to predict whether or not a given person's 8 medical factors point to them having diabetes.

# Implementation

You may notice, we did not do much of a pre-processing.

## Helper Functions

In [14]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [15]:
# Demo:
unique_vals(data, 0)

{0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 17.0}

In [16]:
#count the unique values of labels and store them in a dictionary
def count_values(rows):
    #will return a dictionary with label values as key and frequency as values
    count={}
    #takes whole dataset in as argument
    for row in  rows:
        #traverse on each datapoint
        label=row[-1]
        #labels are in the last column
        #if label is not even once come initialise it
        if label not in count:
            count[label]=0
        #increase the count of present label by 1
        count[label]+=1
    return count 

In [17]:
#Demo
count_values(data)

{1.0: 268, 0.0: 500}

In [18]:
class Question:
    """A Question is used to partition a dataset.
    This class just records a 'column number' (e.g., 1 for glucose) and a
    'column value' (e.g. 148). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """
    #initialise column and value variables->
    #eg->if ques is ->is sepal_length>=1cm then
    #sepal_length==col and 1cm=value
    def __init__(self,column,value):
        self.column=column
        self.value=value
    #it matches wheter the given data is in accordace with the value set or not
    #returns true and false accordingly
    def match(self,data):
        value=data[self.column]
        return value>=self.value
    # This is just a helper method to print
    # the question in a readable format.
    def __repr__(self):
        condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [19]:
#Demo:
#forming a question
Question(0,5)
## it takes column as 0 and value as 5 
q = Question(0,5)
#now it checks wheter the values on 0th column of the 4th datapoint is >= 5 or not
#and returns true or false accordingly
q.match(data[3])

False

## Partition Data w.r.t Question

### <font color = 'red'>TODO</font>: Write code to partition the data using the match function provided in the Question class

In [22]:
#spliting the data based on the respective ques.
def partition(rows,question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    #intialise two seprate lists 
    true_row,false_row=[],[]
    
    for row in rows:
        
        #TODO: your code...
        if(question.match(row)):
            true_row.append(row)
        else:
            false_row.append(row)
    return true_row,false_row

In [23]:
#demo of partition function
#our question is ->
print(Question(0,5))
#t_r represents true_rows and f_r false_rows
t_r, f_r = partition(data,Question(0,5))
#thus t_r will only contain sepal legnth values > 5cm
t_r

Is pregnant >= 5?


[array([  6.   ,   0.   ,  33.6  ,  50.   , 148.   ,  72.   ,   0.627,
          1.   ]),
 array([  8.   ,   0.   ,  23.3  ,  32.   , 183.   ,  64.   ,   0.672,
          1.   ]),
 array([  5.   ,   0.   ,  25.6  ,  30.   , 116.   ,  74.   ,   0.201,
          0.   ]),
 array([ 10.   ,   0.   ,  35.3  ,  29.   , 115.   ,   0.   ,   0.134,
          0.   ]),
 array([  8.   ,   0.   ,   0.   ,  54.   , 125.   ,  96.   ,   0.232,
          1.   ]),
 array([ 10.   ,   0.   ,  38.   ,  34.   , 168.   ,  74.   ,   0.537,
          1.   ]),
 array([ 10.   ,   0.   ,  27.1  ,  57.   , 139.   ,  80.   ,   1.441,
          0.   ]),
 array([  5.   , 175.   ,  25.8  ,  51.   , 166.   ,  72.   ,   0.587,
          1.   ]),
 array([  7.   ,   0.   ,  30.   ,  32.   , 100.   ,   0.   ,   0.484,
          1.   ]),
 array([  7.   ,   0.   ,  29.6  ,  31.   , 107.   ,  74.   ,   0.254,
          1.   ]),
 array([ 8.   ,  0.   , 35.4  , 50.   , 99.   , 84.   ,  0.388,  0.   ]),
 array([  7.   ,   0.   , 

## Gini

### <font color = 'red'>TODO</font>:  Write code to calculate Gini impurity

In [47]:
""""
Calculate the Gini Impurity for a list of rows. There are a few different ways to do this, I thought this one was
the most concise. See:
https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
"""
def gini(rows):
    # TODO: your code...
    count = count_values(rows)
    impurity = 1
    for label in count:
        prob = count[label] / float(len(rows))
        impurity = impurity - prob ** 2
    return impurity

In [48]:
#Demo:
gini(data)

0.45437282986111116

## Entropy

### <font color = 'red'>TODO</font>:  Write code to calculate Entropy

In [26]:
#entropy is basically a measure of chaos-randomness

def entropy(rows):
    #initialise entropy
    entropy=0
    from math import log
    #calculating log(x) in base 2
    
    log2 = lambda x:log(x)/log(2)
    count = count_values(rows)
    
    #storing and traversing the dictionary
    for label in count:
        
        #TODO: your code...
        #Calculate probablity of each unique label and stroe in "p". 
        #Then, Entropy = Entropy - p * log2(p)
        p = count[label] / float(len(rows))
        entropy = entropy - p * log2(p)
        
    return entropy

In [27]:
#Demo: 
entropy(data)

0.9331343166407831

## Weighted Information Gain for Gini and Entropy

Information Gain: The uncertainty of the starting node, minus the weighted impurity of two child nodes.

In [28]:
#weighted info gain
def info_gain_gini(current,left,right):
    #porbab of one branch
    p =float(len(left))/len(left)+len(right)
    #formula for info gian
    return current-p*gini(left)-(1-p)*gini(right)

In [29]:
#weighted info gain
def info_gain_entropy(current,left,right):
    p = float(len(left))/len(left)+len(right)
    return current-p*entropy(left)-(1-p)*entropy(right)

## Calculating Best split

### <font color = 'red'>TODO</font>:  Write code to calculate Best Split

In [30]:
"""Find the best question to ask by iterating over every feature/value and calculating the information gain."""

def best_split(rows):
    
    #initialise best gain and best question
    best_gain=0
    best_question=None
    
    #calculate the current_gain
    current = gini(rows)
    
    #total number of features
    features=len(rows[0])-1
    
    for col in range(features):
        #collects all unique classes for a feature
        values=set([row[col] for row in rows])
        for val in values:
            #TODO: your code ...
            #traverse each unique classs
            #ask the corresponding question
            #devide the data based on that ques
            question = Question(col, val)            
            true_rows, false_rows = partition(rows, question)
            #Continue if len(true_rows == 0) or len(false_rows == 0)
            #Calculate corresponding gain
            if (len(true_rows) == 0 or len(false_rows) == 0):
                continue
            
            gain = info_gain_gini(current, true_rows, false_rows)
            #if gain is > than the best replace
            if gain >= best_gain:
                best_gain,best_question = gain, question
            #iterate through each unique class of each feature and return the best gain and best question     
    return best_gain,best_question

In [31]:
#Demo:
a, b=best_split(data)
'best question initially and info gain by the respective ques'
print(b)
print(a)

Is bp >= 122.0?
349.1688447985703


## Decision Node

In [32]:
#this class represents all nodes in the tree
class DecisionNode:
    def __init__(self,question,true_branch,false_branch):
        #question object stores col and val variables regarding the question of that node
        self.question = question
        #this stores the branch that is true
        self.true_branch = true_branch
        #this stores the false branch
        self.false_branch = false_branch

In [33]:
#Leaf class is the one whichstores leaf of trees
#these are special Leaf Nodes -> on reaching them either
#100% purity is achieved or no features are left to split upon
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """
    def __init__(self,rows):
        #stores unique labels and their values in predictio
        self.predictions=count_values(rows)

## Build Tree (Use Recursion)

### <font color = 'red'>TODO</font>:  Write code to build the tree

In [34]:
def build_tree(rows):
    """Builds the tree.
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    print("In build_tree")
    #takes the whole dataset as argument
    #gets the best gain and best question
    gain, question = best_split(rows)
    
    #if gian=0 i.e. leaf conditions are satisfied
    if gain == 0:
        #make a leaf object and return
        return Leaf(rows)
    
    # If we reach here, we have found a useful feature/value to partition on.
    
    #TODO: your code ...
    # Recursively build the true branch.
    # Recursively build the false branch.
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)

    #returns the root question node storing branches as well as the quesiton
    return DecisionNode(question, true_branch, false_branch)

## Print Tree

In [35]:
def print_tree(node,indentation=""):
    '''printing function'''
    #base case means we have reached the leaf
    #if the node object is of leaf type
    if isinstance(node,Leaf):
        print(indentation+"PREDICTION",node.predictions)
        return 
    #print the question at node
    print(indentation + str(node.question))
    
    #call the function on true branch 
    print(indentation+ "True Branch")
    print_tree(node.true_branch,indentation + " ")
    
    #on flase branch
    print(indentation+ "False Branch")
    print_tree(node.false_branch,indentation + " ")

In [36]:
#building the tree
tree = build_tree(data)

In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In bui

In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In bui

In [37]:
#It won't be very pretty but you get the idea...
print_tree(tree)

Is bp >= 122.0?
True Branch
 PREDICTION {0.0: 1}
False Branch
 Is age >= 81.0?
 True Branch
  PREDICTION {0.0: 1}
 False Branch
  Is age >= 72.0?
  True Branch
   PREDICTION {0.0: 1}
  False Branch
   Is pedigree >= 2.42?
   True Branch
    PREDICTION {1.0: 1}
   False Branch
    Is pedigree >= 2.329?
    True Branch
     PREDICTION {0.0: 1}
    False Branch
     Is pedigree >= 2.288?
     True Branch
      PREDICTION {1.0: 1}
     False Branch
      Is pedigree >= 2.137?
      True Branch
       PREDICTION {1.0: 1}
      False Branch
       Is pedigree >= 1.893?
       True Branch
        PREDICTION {1.0: 1}
       False Branch
        Is pedigree >= 1.781?
        True Branch
         PREDICTION {0.0: 1}
        False Branch
         Is pedigree >= 1.731?
         True Branch
          PREDICTION {0.0: 1}
         False Branch
          Is pedigree >= 1.699?
          True Branch
           PREDICTION {0.0: 1}
          False Branch
           Is pedigree >= 1.698?
           True Br

                                                                                                                                                                                                                                                   False Branch
                                                                                                                                                                                                                                                    Is pregnant >= 1.0?
                                                                                                                                                                                                                                                    True Branch
                                                                                                                                                                                                                                

# Predictions

In [38]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [39]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(data[0], tree)
#######

{1.0: 2}

In [40]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [41]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(data[0], tree))
#######

{1.0: '100%'}

In [42]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(data[1], tree))
#######

{0.0: '100%'}

### Actually doing it:

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)
train_data = np.c_[X_train, y_train]
test_data = np.c_[X_test, y_test]

tree = build_tree(train_data)
print_tree(tree)

for row in test_data:
    actual = row[-1]
    pred = print_leaf(classify(row, tree))
    print("Predicted: " + str(pred) + " Actual: " + str(actual))

In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In bui

In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In build_tree
In bui

                                                                                                                                                                                   True Branch
                                                                                                                                                                                    PREDICTION {0.0: 1}
                                                                                                                                                                                   False Branch
                                                                                                                                                                                    Is insulin >= 204.0?
                                                                                                                                                                                    True Branch
                        

# References:
- https://www.kaggle.com/uciml/pima-indians-diabetes-database#diabetes.csv
- https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
- Class slides about Entropy